In [ ]:
import matplotlib.pyplot as plt # For general plotting

import numpy as np

from scipy.stats import multivariate_normal # MVN not univariate
from sklearn.metrics import confusion_matrix #Confusion matrix

np.set_printoptions(suppress=True)

# Set seed to generate reproducible "pseudo-randomness" (handles scipy's "randomness" too)
np.random.seed(9)

plt.rc('font', size=22)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=18)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=22)   # fontsize of the figure title

**Question-1:**

Given:
1. N = 10000 samples. 
2. class priors probablities: p(L = 0) = 0.65 and p(L = 1) = 0.35
3. Mean vectors:
    1. $$ \mu_{0} = \begin{bmatrix} -1/2 \\ -1/2 \\ -1/2 \end{bmatrix}$$
    2. $$ \mu_{1} = \begin{bmatrix} 1 \\ 1 \\ 1 \end{bmatrix}$$
4. Covariance Matrices:
    1. $$ \Sigma_{0} = \begin{bmatrix} 1 & -0.5 & 0.3 \\ -0.5 & 1 & -0.5 \\ 0.3 & -0.5 & 1 \end{bmatrix}$$
    1. $$ \Sigma_{1} = \begin{bmatrix} 1 & 0.3 & -0.2 \\ 0.3 & 1 & 0.3 \\ 0.2 & 0.3 & 1 \end{bmatrix}$$
    
Generate N independent and identically distributed random 3-D samples from the given Gaussian Probablity Density functions with given class priors probablities.

In [ ]:
# Number of samples to draw from each distribution
N = 10000

mu = np.array([[-1/2, -1/2, -1/2],
               [1, 1, 1]])  # Gaussian distributions means
Sigma = np.array([[[1, -0.5, 0.3],
                   [-0.5, 1, -0.5],
                   [0.3, -0.5, 1]],
                  [[1, 0.3, -0.2],
                   [0.3, 1, 0.3], 
                   [-0.2, 0.3, 1]]])  # Gaussian distributions covariance matrices


# Determine dimensionality from PDF parameters
n = mu.shape[1]

# Class priors
priors = np.array([0.65, 0.35])  
C = len(priors)

# Decide randomly which samples will come from each component
labels = np.random.rand(N) >= priors[0]
L = np.array(range(C))
Nl = np.array([sum(labels == i) for i in L])

# Draw samples from each class pdf
X = np.zeros((N, n))
X[labels == 0, :] =  multivariate_normal.rvs(mu[0], Sigma[0], Nl[0])
X[labels == 1, :] =  multivariate_normal.rvs(mu[1], Sigma[1], Nl[1])


# Plot the original data and their true labels
fig = plt.figure(figsize=(10, 10))
axis = plt.axes(projection='3d')
axis.plot(X[labels==0, 0], X[labels==0, 1], X[labels==0, 2],  'ro', label="Class 0")
axis.plot(X[labels==1, 0], X[labels==1, 1], X[labels==1, 2], 'go', label="Class 1")

plt.legend()
axis.set_xlabel(r"$x_1$")
axis.set_ylabel(r"$x_2$")
axis.set_zticks([-4, -2, 0, 2, 4])
axis.set_zlabel(r"$x_3$")
plt.title("Data and True Class Labels")
plt.tight_layout()
plt.show()

print("Count of Class-0 Samples", Nl[0])
print("Count of Class-1 Samples", Nl[1])

**PART- A: Question-1**

Given:
$${p(x |L=1) \over p(x |(L=0)} > \gamma$$ 

Solution:
ERM CLassifier: with 2 Gaussian Mixture model class conditionals we can derive the above equation to:

$$ \implies {p(x |L=1) \over p(x |(L=0)} = \frac {\sum_{i=1}^ {\mu_{1}} \alpha_{1i} g(x_{i}\mu_{1i}\Sigma_{1i})}{\sum_{i=1}^ {\mu_{0}} \alpha_{0i} g(x_{i}\mu_{0i}\Sigma_{0i}))}$$
$$ \implies {p(x |L=1) \over p(x |(L=0)} = \frac {g(x|m1,C1)}{g(x|m0,C0)} $$

where g(x|m,C) is a multi variate gaussian probablity density function with mean vector m and covariance matrix C.
$$ \implies {p(x |L=1) \over p(x |(L=0)} = \frac {g(x|m1,C1)}{g(x|m0,C0)}  > \gamma$$
$$ \implies {p(x |L=1) \over p(x |(L=0)} = \frac {g(x|m1,C1)}{g(x|m0,C0)}  > \gamma = \frac {p(x|L=0)}{p(x|L=1)} * \frac {\delta_{01}-\delta_{00}}{\delta_{10}-\delta_{11}}$$

If we consider 1-0 loss matrix then minimum probablity will be:
$$ \gamma = \frac {p(L=0)}{p(L=1)} $$
$$ \implies \gamma = \frac {0.65}{0.35} $$
$$ \implies \gamma = 1.8572 $$
This is the $\gamma$ value for the lowest likelihood of mistake.

In [ ]:
# Expected Risk Minimization Classifier (using true model parameters)
class_conditional_likelihoods = np.array([multivariate_normal.pdf(X, mu[i], Sigma[i]) for i in L])
# Discriminant score is LHS of likelihood-ratio test (LRT) 
discriminant_score_erm = np.log(class_conditional_likelihoods[1]) - np.log(class_conditional_likelihoods[0])

# Gamma threshold for MAP decision rule (same gamma on priors only; 0-1 loss simplification)
gamma_map = priors[0] / priors[1]
decisions_map = discriminant_score_erm >= np.log(gamma_map)

# Get indices and probability estimates of the four decision scenarios:
# (true negative, false positive, false negative, true positive)

# True Negative Probability
ind_00_map = np.argwhere((decisions_map==0) & (labels==0))
p_00_map = len(ind_00_map) / Nl[0]
# False Positive Probability
ind_10_map = np.argwhere((decisions_map==1) & (labels==0))
p_10_map = len(ind_10_map) / Nl[0]
# False Negative Probability
ind_01_map = np.argwhere((decisions_map==0) & (labels==1))
p_01_map = len(ind_01_map) / Nl[1]
# True Positive Probability
ind_11_map = np.argwhere((decisions_map==1) & (labels==1))
p_11_map = len(ind_11_map) / Nl[1]

# Probability of error for MAP classifier, empirically estimated
prob_error_erm = np.array((p_10_map, p_01_map)).dot(Nl.T / N)
print("Smallest Pr(error) for ERM = {}".format(prob_error_erm))

#Code to flatten the list of lists and storing using numpy array in order to plot in 3D.
flat_list =  [item for sublist in ind_00_map for item in sublist]
ind_00_map_flat_list = np.array(flat_list)

flat_list =  [item for sublist in ind_10_map for item in sublist]
ind_10_map_flat_list = np.array(flat_list)

flat_list =  [item for sublist in ind_01_map for item in sublist]
ind_01_map_flat_list = np.array(flat_list)

flat_list =  [item for sublist in ind_11_map for item in sublist]
ind_11_map_flat_list = np.array(flat_list)


# Plotting in 3D and displaying the MAP Decisions.
fig = plt.figure(figsize=(10, 10))
axis = plt.axes(projection='3d')
axis.plot(X[ind_00_map_flat_list, 0], X[ind_00_map_flat_list, 1], X[ind_00_map_flat_list, 2],  
          'og', label="Class 0 - Correct")
axis.plot(X[ind_10_map_flat_list, 0], X[ind_10_map_flat_list, 1], X[ind_10_map_flat_list, 2],  
          'or', label="Class 0 - Incorrect")
axis.plot(X[ind_01_map_flat_list, 0], X[ind_01_map_flat_list, 1], X[ind_01_map_flat_list, 2],  
          '^r', label="Class 1 - Incorrect")
axis.plot(X[ind_11_map_flat_list, 0], X[ind_11_map_flat_list, 1], X[ind_11_map_flat_list, 2],  
          '^g', label="Class 1 - Correct")

plt.legend()
axis.set_xlabel(r"$x_1$")
axis.set_ylabel(r"$x_2$")
axis.set_zticks([-4, -2, 0, 2, 4])
axis.set_zlabel(r"$x_3$")
plt.title("MAP Decisions (RED incorrect)")
plt.tight_layout()
plt.show()


**PART-A:Question-2, 3**

In [ ]:
from sys import float_info # Threshold smallest positive floating value

# Generate ROC curve samples
def estimate_roc(discriminant_score, label):
    Nlabels = np.array((sum(label == 0), sum(label == 1)))

    # Sorting necessary so the resulting FPR and TPR axes plot threshold probabilities in order as a line
    sorted_score = sorted(discriminant_score)

    # Use gamma values that will account for every possible classification split
    gammas = ([sorted_score[0] - float_info.epsilon] +
              sorted_score +
              [sorted_score[-1] + float_info.epsilon])

    # Calculate the decision label for each observation for each gamma
    decisions = [discriminant_score >= g for g in gammas]

    ind10 = [np.argwhere((d==1) & (label==0)) for d in decisions]
    p10 = [len(inds)/Nlabels[0] for inds in ind10]
    ind11 = [np.argwhere((d==1) & (label==1)) for d in decisions]
    p11 = [len(inds)/Nlabels[1] for inds in ind11]

    # ROC has FPR on the x-axis and TPR on the y-axis
    roc = np.array((p10, p11))

    return roc, gammas

**Construct the ROC for ERM by changing log(gamma)**

In [ ]:
# Construct the ROC for ERM by changing log(gamma)
roc_erm, _ = estimate_roc(discriminant_score_erm, labels)
roc_map = np.array((p_10_map, p_11_map))

plt.ioff() # These are Jupyter only lines to avoid showing the figure when I don't want
fig_roc, ax_roc = plt.subplots(figsize=(10, 10))
plt.ion() # Re-activate "interactive" mode

ax_roc.plot(roc_erm[0], roc_erm[1])
ax_roc.plot(roc_map[0], roc_map[1], 'rx', label="Minimum Pr(error) MAP", markersize=29)
ax_roc.legend()
ax_roc.set_xlabel(r"Probability of false alarm $p(D=1|L=0)$")
ax_roc.set_ylabel(r"Probability of correct decision $p(D=1|L=1)$")
plt.grid(True)
display(fig_roc) # Display as .png

fig_roc;

In [ ]:
print("For ERM with correct distribution data, the Minimum Pr(error)",format(prob_error_erm))

**PART-B**

Given:
1. Class conditional pdfs are both Gaussian with the correct True means - Same mean vectors = same $\mu$
2. Mean vectors:
    1. $$ \mu_{0} = \begin{bmatrix} -1/2 \\ -1/2 \\ -1/2 \end{bmatrix}$$
    2. $$ \mu_{1} = \begin{bmatrix} 1 \\ 1 \\ 1 \end{bmatrix}$$
3. Class conditional pdfs are both Gaussian with the incorrect covariance matrices = Indetity Matrices
4. Covariance Matrices:
    1. $$ \Sigma_{0} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix}$$
    2. $$ \Sigma_{1} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix}$$

In [ ]:
Sigma_Identity = np.array([[[1, 0, 0],
                   [0, 1, 0],
                   [0, 0, 1]],
                  [[1, 0, 0],
                   [0, 1, 0], 
                   [0, 0, 1]]])  # Gaussian distributions covariance matrices

# Expected Risk Minimization Classifier (using true model parameters)
class_conditional_likelihoods = np.array([multivariate_normal.pdf(X, mu[i], Sigma_Identity[i]) for i in L])
# Discriminant score is LHS of likelihood-ratio test (LRT) 
discriminant_score_erm = np.log(class_conditional_likelihoods[1]) - np.log(class_conditional_likelihoods[0])

# Gamma threshold for MAP decision rule (same gamma on priors only; 0-1 loss simplification)
gamma_map = priors[0] / priors[1]
decisions_map = discriminant_score_erm >= np.log(gamma_map)

# Get indices and probability estimates of the four decision scenarios:
# (true negative, false positive, false negative, true positive)

# True Negative Probability
ind_00_map = np.argwhere((decisions_map==0) & (labels==0))
p_00_map = len(ind_00_map) / Nl[0]
# False Positive Probability
ind_10_map = np.argwhere((decisions_map==1) & (labels==0))
p_10_map = len(ind_10_map) / Nl[0]
# False Negative Probability
ind_01_map = np.argwhere((decisions_map==0) & (labels==1))
p_01_map = len(ind_01_map) / Nl[1]
# True Positive Probability
ind_11_map = np.argwhere((decisions_map==1) & (labels==1))
p_11_map = len(ind_11_map) / Nl[1]

# Probability of error for MAP classifier, empirically estimated
prob_error_erm = np.array((p_10_map, p_01_map)).dot(Nl.T / N)
print("Smallest Pr(error) for ERM = {}".format(prob_error_erm))

#Code to flatten the list of lists and storing using numpy array in order to plot in 3D.
flat_list =  [item for sublist in ind_00_map for item in sublist]
ind_00_map_flat_list = np.array(flat_list)

flat_list =  [item for sublist in ind_10_map for item in sublist]
ind_10_map_flat_list = np.array(flat_list)

flat_list =  [item for sublist in ind_01_map for item in sublist]
ind_01_map_flat_list = np.array(flat_list)

flat_list =  [item for sublist in ind_11_map for item in sublist]
ind_11_map_flat_list = np.array(flat_list)


# Plotting in 3D and displaying the MAP Decisions.
fig = plt.figure(figsize=(10, 10))
axis = plt.axes(projection='3d')
axis.plot(X[ind_00_map_flat_list, 0], X[ind_00_map_flat_list, 1], X[ind_00_map_flat_list, 2],  
          'og', label="Class 0 - Correct")
axis.plot(X[ind_10_map_flat_list, 0], X[ind_10_map_flat_list, 1], X[ind_10_map_flat_list, 2],  
          'or', label="Class 0 - Incorrect")
axis.plot(X[ind_01_map_flat_list, 0], X[ind_01_map_flat_list, 1], X[ind_01_map_flat_list, 2],  
          '^r', label="Class 1 - Incorrect")
axis.plot(X[ind_11_map_flat_list, 0], X[ind_11_map_flat_list, 1], X[ind_11_map_flat_list, 2],  
          '^g', label="Class 1 - Correct")

plt.legend()
axis.set_xlabel(r"$x_1$")
axis.set_ylabel(r"$x_2$")
axis.set_zticks([-4, -2, 0, 2, 4])
axis.set_zlabel(r"$x_3$")
plt.title("MAP Decisions (RED incorrect)")
plt.tight_layout()
plt.show()


In [ ]:
from sys import float_info # Threshold smallest positive floating value

# Generate ROC curve samples
def estimate_roc(discriminant_score, label):
    Nlabels = np.array((sum(label == 0), sum(label == 1)))

    # Sorting necessary so the resulting FPR and TPR axes plot threshold probabilities in order as a line
    sorted_score = sorted(discriminant_score)

    # Use gamma values that will account for every possible classification split
    gammas = ([sorted_score[0] - float_info.epsilon] +
              sorted_score +
              [sorted_score[-1] + float_info.epsilon])

    # Calculate the decision label for each observation for each gamma
    decisions = [discriminant_score >= g for g in gammas]

    ind10 = [np.argwhere((d==1) & (label==0)) for d in decisions]
    p10 = [len(inds)/Nlabels[0] for inds in ind10]
    ind11 = [np.argwhere((d==1) & (label==1)) for d in decisions]
    p11 = [len(inds)/Nlabels[1] for inds in ind11]

    # ROC has FPR on the x-axis and TPR on the y-axis
    roc = np.array((p10, p11))

    return roc, gammas

In [ ]:
# Construct the ROC for ERM by changing log(gamma)
roc_erm, _ = estimate_roc(discriminant_score_erm, labels)
roc_map = np.array((p_10_map, p_11_map))

plt.ioff() # These are Jupyter only lines to avoid showing the figure when I don't want
fig_roc, ax_roc = plt.subplots(figsize=(10, 10))
plt.ion() # Re-activate "interactive" mode

ax_roc.plot(roc_erm[0], roc_erm[1])
ax_roc.plot(roc_map[0], roc_map[1], 'rx', label="Minimum Pr(error) MAP using Naive Bayesian Approach", markersize=29)
ax_roc.legend()
ax_roc.set_xlabel(r"Probability of false alarm $p(D=1|L=0)$")
ax_roc.set_ylabel(r"Probability of correct decision $p(D=1|L=1)$")
plt.grid(True)
display(fig_roc) # Display as .png

fig_roc;

In [ ]:
print("For ERM with incorrect distribution data using Naive bayes approach, the Minimum Pr(error)",format(prob_error_erm))

**PART-C Fisher's Linear Discriminant Analysis (LDA)**

In [ ]:
def perform_lda(X, mu, Sigma, C=2):
    """  Fisher's Linear Discriminant Analysis (LDA) on data from two classes (C=2).

    In practice the mean and covariance parameters would be estimated from training samples.
    
    Args:
        X: Real-valued matrix of samples with shape [N, n], N for sample count and n for dimensionality.
        mu: Mean vector [C, n].
        Sigma: Covariance matrices [C, n, n].

    Returns:
        w: Fisher's LDA project vector, shape [n, 1].
        z: Scalar LDA projections of input samples, shape [N, 1].
    """

    mu = np.array([mu[i].reshape(-1, 1) for i in range(C)])
    cov = np.array([Sigma[i].T for i in range(C)])

    # Determine between class and within class scatter matrix
    Sb = (mu[1] - mu[0]).dot((mu[1] - mu[0]).T)
    Sw = cov[0] + cov[1]

    # Regular eigenvector problem for matrix Sw^-1 Sb
    lambdas, U = np.linalg.eig(np.linalg.inv(Sw).dot(Sb))
    # Get the indices from sorting lambdas in order of increasing value, with ::-1 slicing to then reverse order
    idx = lambdas.argsort()[::-1]

    # Extract corresponding sorted eigenvectors
    U = U[:, idx]

    # First eigenvector is now associated with the maximum eigenvalue, mean it is our LDA solution weight vector
    w = U[:, 0]

    # Scalar LDA projections in matrix form
    z = X.dot(w)

    return w, z

In [ ]:
# Fisher LDA Classifer (using true model parameters)
weight, discriminant_score_lda = perform_lda(X, mu, Sigma)

# Estimate the ROC curve for this LDA classifier
roc_lda, gamma_lda = estimate_roc(discriminant_score_lda, labels)

# ROC returns FPR vs TPR, but prob error needs FNR so take 1-TPR
prob_error_lda = np.array((roc_lda[0,:], 1 - roc_lda[1,:])).T.dot(Nl.T / N)

# Min prob error
min_prob_error_lda = np.min(prob_error_lda)
min_ind = np.argmin(prob_error_lda)

# Display the estimated ROC curve for LDA and indicate the operating points
# with smallest empirical error probability estimates (could be multiple)
ax_roc.plot(roc_lda[0], roc_lda[1], 'b:')
ax_roc.plot(roc_lda[0, min_ind], roc_lda[1, min_ind], 'r.', label="Minimum Pr(error) LDA", markersize=16)
ax_roc.set_title("ROC Curves for ERM and LDA")
ax_roc.legend()

plt.show()
display(fig_roc)
fig_roc;

print("Weight Vector", weight)

In [ ]:
# Use min-error threshold
decisions_lda = discriminant_score_lda >= gamma_lda[min_ind]

# Get indices and probability estimates of the four decision scenarios:
# (true negative, false positive, false negative, true positive)

# True Negative Probability
ind_00_lda = np.argwhere((decisions_lda==0) & (labels==0))
p_00_lda = len(ind_00_lda) / Nl[0]
# False Positive Probability
ind_10_lda = np.argwhere((decisions_lda==1) & (labels==0))
p_10_lda = len(ind_10_lda) / Nl[0]
# False Negative Probability
ind_01_lda = np.argwhere((decisions_lda==0) & (labels==1))
p_01_lda = len(ind_01_lda) / Nl[1]
# True Positive Probability
ind_11_lda = np.argwhere((decisions_lda==1) & (labels==1))
p_11_lda = len(ind_11_lda) / Nl[1]

#Code to flatten the list of lists and storing using numpy array in order to plot in 3D.
flat_list =  [item for sublist in ind_00_lda for item in sublist]
ind_00_lda_flat_list = np.array(flat_list).astype(int)

flat_list =  [item for sublist in ind_10_lda for item in sublist]
ind_10_lda_flat_list = np.array(flat_list).astype(int)

flat_list =  [item for sublist in ind_01_lda for item in sublist]
ind_01_lda_flat_list = np.array(flat_list).astype(int)

flat_list =  [item for sublist in ind_11_lda for item in sublist]
ind_11_lda_flat_list = np.array(flat_list).astype(int)


# Plotting in 3D and displaying the MAP Decisions.
fig = plt.figure(figsize=(10, 10))
axis = plt.axes(projection='3d')
axis.plot(X[ind_00_lda_flat_list, 0], X[ind_00_lda_flat_list, 1], X[ind_00_lda_flat_list, 2],  
          'og', label="Class 0 - Correct")
axis.plot(X[ind_10_lda_flat_list, 0], X[ind_10_lda_flat_list, 1], X[ind_10_lda_flat_list, 2],  
          'or', label="Class 0 - Incorrect")
axis.plot(X[ind_01_lda_flat_list, 0], X[ind_01_lda_flat_list, 1], X[ind_01_lda_flat_list, 2],  
          '^r', label="Class 1 - Incorrect")
axis.plot(X[ind_11_lda_flat_list, 0], X[ind_11_lda_flat_list, 1], X[ind_11_lda_flat_list, 2],  
          '^g', label="Class 1 - Correct")

plt.legend()
axis.set_xlabel(r"$x_1$")
axis.set_ylabel(r"$x_2$")
axis.set_zticks([-4, -2, 0, 2, 4])
axis.set_zlabel(r"$x_3$")
plt.title("LDA Decisions (RED incorrect)")
plt.tight_layout()
plt.show()


In [ ]:
print("Smallest P(error) for LDA = {}".format(min_prob_error_lda))

Findings:

1. For Part-A, The initial implementation of ERM (or MAP) mismatches with the Naive Bayesian. It is clear from the variations in minimum probability of errors. The ROC curves are similar with slight deviation.

2. For Part-C, We anticipated that the MAP classifier's error probability would be lower than that of all other classifiers, including Fisher's LDA, because it is built to reduce probability error (when used with correct class conditional likelihoods and class priors). This theoretical result is supported by the numerical experiment.

3. We also note that because the dataset is random, it is possible that the empirical error probability estimate of the LDA classifier will occasionally (less frequently for larger N) be lower than that of the MAP classifier. This is because error probability estimates are subject to random sample variations.

4. Also take note of how the LDA classifier's ROC curve in this instance resembles both the ERM (MAP) classifier and the Naive Bayesian technique used in the ERM with incorrect data.

5. The same steps as in Part A are used to calculate the gammas and minimum error probability. There is only a small discrepancy between these values and the LDA and Naive Bayesian Classifier. The ROC curve doesn't negatively affected by the model mismatch, and both the theoretical and experimental probability of mistakes actually decreased significantly.

**Question-2**
**PART-A.1**

In [ ]:
# Widget to manipulate plots in Jupyter notebooks
import matplotlib.pyplot as plt # For general plotting

import numpy as np

from scipy.stats import multivariate_normal # MVN not univariate

np.set_printoptions(suppress=True)

# Set seed to generate reproducible "pseudo-randomness" (handles scipy's "randomness" too)
np.random.seed(7)

plt.rc('font', size=22)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=18)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=22)   # fontsize of the figure title

## Gaussian Mixture Model Setting

Mixture models have the form:

$$ p(\mathbf{x}) = \sum_{c=1}^C a_c p_c(\mathbf{x}). $$

These models are a linear combination of multiple component distributions, where $p_c(\mathbf{x})$ is the $c^{th}$ probability density function (pdf), and $a_c$ is the $c^{th}$ "mixture weight", such that both $0 \leq a_c \leq 1$ and $\sum_{c=1}^C a_c = 1$ are satisfied.

Let's look at the concrete example of a Gaussian Mixture Model (GMM), where $p_c(\mathbf{x}) = \mathcal{N}(\mathbf{x} \,|\, \boldsymbol{\mu}_c, \boldsymbol{\Sigma}_c)$ for $c \in \{1, ..., C\}$, with each $c^{th}$ component distribution represented as a multivariate Gaussian. Effectively, we are "choosing" a Gaussian pdf to sample from based on the weights $a_c$.

Below provides an example of how to draw samples using the SciPy library, notably the `scipy.stats` module and its `multivariate_normal` class. It also defines the dataset setting for our ERM demonstration, which consists of the following conditional log-likelihoods for 4 classes with true labels $L$:

$$ p(\mathbf{x} \,|\, L=1) = \mathcal{N}(\mathbf{x} \,|\, \boldsymbol{\mu}_{1}, \boldsymbol{\Sigma}_{1}),$$
$$ p(\mathbf{x} \,|\, L=2) = \mathcal{N}(\mathbf{x} \,|\, \boldsymbol{\mu}_{2}, \boldsymbol{\Sigma}_{2}),$$
$$ p(\mathbf{x} \,|\, L=3) = \mathcal{N}(\mathbf{x} \,|\, \boldsymbol{\mu}_{3}, \boldsymbol{\Sigma}_{3}),$$
$$ p(\mathbf{x} \,|\, L=4) = \mathcal{N}(\mathbf{x} \,|\, \boldsymbol{\mu}_{4}, \boldsymbol{\Sigma}_{4}),$$


Given:
1. Class priors as p(L=0) = 0.2, p(L=1) = 0.25, p(L=2) = 0.25, p(L=3) = 0.3
2. Mean vectors are equally spaces along the line.
3. Covariance matrices are scaled versions of identity matrix.


\begin{equation*}
    \boldsymbol{\mu}_1=\begin{bmatrix} 1\\0 \end{bmatrix}, ~~~
    \boldsymbol{\mu}_2=\begin{bmatrix} 5\\0 \end{bmatrix}, ~~~
    \boldsymbol{\mu}_3=\begin{bmatrix} 9\\0 \end{bmatrix}, ~~~
    \boldsymbol{\mu}_4=\begin{bmatrix} 13\\0 \end{bmatrix} ~~~ \\
    \boldsymbol{\Sigma}_1=\begin{bmatrix}1 & 0 \\0 & 1 \end{bmatrix}, ~~~
    \boldsymbol{\Sigma}_2=\begin{bmatrix}2 & 0 \\0 & 2 \end{bmatrix}, ~~~
    \boldsymbol{\Sigma}_3=\begin{bmatrix}3 & 0 \\0 & 3 \end{bmatrix},
    \boldsymbol{\Sigma}_4=\begin{bmatrix}4 & 0 \\0 & 4 \end{bmatrix}
\end{equation*}


In [ ]:
# Number of samples to draw from each distribution
N = 10000

# Likelihood of each distribution to be selected AND class priors!!!
priors = np.array([0.2, 0.25, 0.25, 0.3])  
# Determine number of classes/mixture components
C = len(priors)
mu = np.array([[1, 0],
               [5, 0],
               [9, 0],
               [13, 0]])  # Gaussian distributions means
Sigma = np.array([[[1, 0],
                   [0, 1]],
                  [[2, 0],
                   [0, 2]],
                  [[3, 0],
                   [0, 3]],
                  [[4, 0],
                   [0, 4]]])  # Gaussian distributions covariance matrices

# Determine dimensionality from mixture PDF parameters
n = mu.shape[1]
# Output samples and labels
X = np.zeros([N, n])
labels = np.zeros(N)

# Decide randomly which samples will come from each component u_i ~ Uniform(0, 1) for i = 1, ..., N (or 0, ... , N-1 in code)
u = np.random.rand(N)
# Determine the thresholds based on the mixture weights/priors for the GMM, which need to sum up to 1
thresholds = np.cumsum(priors)

for c in range(C):
    c_ind = np.argwhere(u <= thresholds[c])[:, 0]  # Get randomly sampled indices for this component
    c_N = len(c_ind)  # No. of samples in this component
    labels[c_ind] = c * np.ones(c_N)
    u[c_ind] = 1.1 * np.ones(c_N)  # Multiply by 1.1 to fail <= thresholds and thus not reuse samples
    X[c_ind, :] =  multivariate_normal.rvs(mu[c], Sigma[c], c_N)

# Plot the original data and their true labels
plt.figure(figsize=(10, 10))
plt.plot(X[labels==0, 0], X[labels==0, 1], 'bo', label="Class 1")
plt.plot(X[labels==1, 0], X[labels==1, 1], 'rx', label="Class 2");
plt.plot(X[labels==2, 0], X[labels==2, 1], 'g+', label="Class 3");
plt.plot(X[labels==3, 0], X[labels==3, 1], 'y^', label="Class 4");
plt.legend()
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$")
plt.title("Data and True Labels")
plt.tight_layout()
plt.show()

**PART-A.2 and PART-A.3**

## ERM Classification using True Knowledge of PDF

For this demo of ERM, we are specifically going to use <b>0-1 loss</b>, such that the decision rule we wish to derive achieves minimum probability of error, i.e. the <b> maximum a posteriori (MAP)</b> classification rule. We will implement this classifier with the true distribution knowledge we have outlined above, and count samples per decision-label pair to produce a confusion matrix with entries $p(D=i\,|\,L=j)$ for $i,j \in \{1, 2, 3, 4\}$ 

In [ ]:
L = np.array(range(C))  # 0-(C-1)

The decision rule that achieves minimum probability of error uses a loss matrix $\mathbf{\Lambda}$ with entries $\lambda_{ij} = 1 - \delta_{ij}$ where $\lambda_{ij}$ is the loss associated with deciding class label $i$ given $\mathbf{x}$ comes from class label $j$ and $\delta_{ij}$ is the Kronecker delta: 

$$\lambda_{ij} =
    \begin{cases}
        0  & \text{if} \; i = j \\
         1 & \text{if} \; i \neq j
    \end{cases}
$$

From this, we can define the following 0-1 loss matrix:

$$ \mathbf{\Lambda} = \begin{bmatrix} 0 & 1 & 1 & 1 \\ 1 & 0 & 1 & 1 \\ 1 & 1 & 0 &1 \\ 1 & 1 & 1 &0 \end{bmatrix}. $$

In [ ]:
# We are going to use a 0-1 loss matrix for this problem
Lambda = np.ones((C, C)) - np.identity(C)
print(Lambda)

For a given $\mathbf{x}$, we want to choose a class label $i$ which minimizes risk (or loss) associated with choosing this class label. Hence, we need a decision rule $D(\mathbf{x})$, which is a function $D : \mathbb{R}^n \rightarrow {1, \ldots, C} \in \mathbb{Z}$ that tells us which action to take or decision to make for every possible observation.

We know that the ERM decision rule $D(\mathbf{x})$ is based on minimizing conditional risk, defined as:

$$ D(\mathbf{x}) = \mathop{\rm argmin}_{i\in \{1, 2, 3, 4\}} R(D = i \, | \, \mathbf{x}) = \mathop{\rm argmin}_{i\in \{1, 2, 3, 4\}} \sum_{j=1}^4 \lambda_{ij} p(L = j \, | \, \mathbf{x}).$$

In the case of a 0-1 loss function $\lambda_{ij}$, then:

\begin{align*}
R(D = i \, | \, \mathbf{x}) 
& = \sum_{j=1}^4 \lambda_{ij} p(L = j \, | \, \mathbf{x}) \\
& = \sum_{j \neq i} p(L = j \, | \, \mathbf{x}) \\
& = 1 - p(L = i \, | \, \mathbf{x}).
\end{align*}

As a result, the decision rule $D(\mathbf{x})$ ends up <b>minimizing the average probability of error</b> (misclassification rate or posterior expected loss):

\begin{align*}
 D(\mathbf{x}) & = \mathop{\rm argmin}_{i\in \{1, 2, 3, 4\}} R(D = i \, | \, \mathbf{x}) \\ 
               & = \mathop{\rm argmin}_{i\in \{1, 2, 3, 4\}} 1 - p(L = i \, | \, \mathbf{x}) \\
               & = \mathop{\rm argmax}_{i\in \{1, 2, 3, 4\}} p(L = i \, | \, \mathbf{x}),
\end{align*}

or equivalently, as shown in the last step, computing the <b>MAP estimate</b>.

While we have just derived the ERM decision rule <i>without</i> needing a loss matrix $\boldsymbol{\Lambda}$ due to the 0-1 loss setting, I will still proceed with its application for sake of generality.

In particular, I wish to break down $D(\mathbf{x})$ in the code by first converting the risk $R(D = i \, | \, \mathbf{x})$ of choosing decision $i$ given observation $\mathbf{x}$ into matrix form (see extra notes):

$$ \begin{bmatrix} R(D=1\, | \,\mathbf{x}) \\  R(D=2\, | \,\mathbf{x}) \\ R(D=3\, | \,\mathbf{x} \\  R(D=4\, | \,\mathbf{x}) \end{bmatrix} = \mathbf{\Lambda} \begin{bmatrix} p(L=1\, | \,\mathbf{x}) \\ p(L=2\, | \,\mathbf{x}) \\ p(L=3\, | \,\mathbf{x} \\ p(L=4\, | \,\mathbf{x}) \end{bmatrix} = \mathbf{\Lambda} \, \text{diag}\big(p(L=1), p(L=2), p(L=3), p(L=4) \big) \begin{bmatrix} p(\mathbf{x}\, | \,L=1) \\ p(\mathbf{x}\, | \,L=2) \\ p(\mathbf{x}\, | \,L=3) \\ p(\mathbf{x}\, | \,L=4) \end{bmatrix}.$$

The LHS of this equality is the risk across all decision options, and the RHS of the equality is the loss matrix and class posteriors, which have been expanded using Bayes rule into likelihood and prior terms. Therefore we can proceed in the code by computing the class posteriors from the priors and conditional likelihoods:

In [ ]:
# Calculate class-conditional likelihoods p(x|L=j) for each label of the N observations
class_cond_likelihoods = np.array([multivariate_normal.pdf(X, mu[j], Sigma[j]) for j in L])
class_priors = np.diag(priors)
print("Class condition likelihoods",class_cond_likelihoods.shape)
print("Clas priors", class_priors.shape)
# Matrix of posteriors, shaped [C, N]
class_posteriors = class_priors.dot(class_cond_likelihoods)
print("Class posteriors", class_posteriors)

And then the conditional risk scores using $\boldsymbol{\Lambda}$, which again I emphasize is not required for 0-1 loss but demonstrates the full ERM framework:

In [ ]:
# We want to create the risk matrix of size C x N
cond_risk = Lambda.dot(class_posteriors)
print("Condition Risk", cond_risk)

This finally leaves us with our decision vector for all observations:

$$ D(\mathbf{x}) = \mathop{\rm argmin}_{i\in \{1, 2, 3, 4\}} R(D = i \, | \, \mathbf{x}). $$

In [ ]:
# Get the decision for each column in risk_mat
decisions = np.argmin(cond_risk, axis=0)
print("Decisions", decisions.shape)

We can now yield the following <b>normalized</b> confusion matrix over the true columns (class sample counts):
    
$$ \begin{bmatrix} p(D=1\, | \,L=1) & \cdots & p(D=4\, | \,L=1) \\ \vdots & \ddots & \vdots \\  p(D=1\, | \,L=4) & \cdots & p(D=4\, | \,L=4)  \end{bmatrix}. $$

And reminder that the <b>probability of error</b> is:

$$ \text{Pr}(\text{error}) = 1 - \text{Pr}(\text{correct}) = 1 - \frac{1}{N}\sum_i^4 N_i p(D = i \,|\, L = i),$$

with $N_i$ as the number of samples belonging to class $i$. So the final minimum probability of error estimate using our MAP classifier is computed by:

In [ ]:
# Plot for decisions vs true labels
fig = plt.figure(figsize=(10, 10))
marker_shapes = 'o.^*' # Accomodates up to C=5
marker_colors = 'gr'

# Get sample class counts
sample_class_counts = np.array([sum(labels == j) for j in L])

# Confusion matrix compute from scratch
conf_mat = np.zeros((C, C))
for i in L: # Each decision option
    for j in L: # Each class label
        ind_ij = np.argwhere((decisions==i) & (labels==j))
        conf_mat[i, j] = len(ind_ij) / sample_class_counts[j] # Average over class sample count

        # True label = Marker shape; Decision = Marker Color
        marker = marker_shapes[j]
        
        

        if i != j:
            plt.plot(X[ind_ij, 0], X[ind_ij, 1], marker+'r')
        else:
            plt.plot(X[ind_ij, 0], X[ind_ij, 1], marker+'g')
            
print("Confusion matrix:")
print(conf_mat)

# # Can also compute the normalized confusion matrix from scikit-learn's method
# from sklearn.metrics import confusion_matrix
# print("Normalized Sklearn Confusion Matrix (column basis):")
# conf_mat = confusion_matrix(decisions, labels, normalize='pred')
# print(conf_mat)

prob_error = 1 - np.diag(conf_mat).dot(sample_class_counts) / N
print("Minimum Probability of Error: ", prob_error)

plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$")
plt.title("Minimum Probability of Error:  {:.3f}".format(prob_error))
plt.show()

**Question-2:Part-B**

$$ \mathbf{\Lambda} = \begin{bmatrix} 0 & 1 & 2 & 3 \\ 1 & 0 & 1 & 2 \\ 2 & 1 & 0 &1 \\ 3 & 2 & 1 &0 \end{bmatrix}. $$
Repating the same with the above loss values.

In [ ]:
# We are going to use a 0-1 loss matrix for this problem
Lambda = np.array([[0.0,1.0,2.0,3.0],[1.0,0.0,1.0,2.0],[2.0,1.0,0.0,1.0],[3.0,2.0,1.0,0.0]])
print("Lambda",Lambda)

In [ ]:
# Calculate class-conditional likelihoods p(x|L=j) for each label of the N observations
class_cond_likelihoods = np.array([multivariate_normal.pdf(X, mu[j], Sigma[j]) for j in L])
class_priors = np.diag(priors)
print("class_cond_likelihoods", class_cond_likelihoods.shape)
print("class_priors", class_priors.shape)
# Matrix of posteriors, shaped [C, N]
class_posteriors = class_priors.dot(class_cond_likelihoods)
print("class_posteriors", class_posteriors)

In [ ]:
# We want to create the risk matrix of size C x N
cond_risk = Lambda.dot(class_posteriors)
print("cond_risk", cond_risk)

In [ ]:
# Get the decision for each column in risk_mat
decisions_new = np.argmin(cond_risk, axis=0)
print("decisions_new", decisions_new.shape)

In [ ]:
# Plot for decisions vs true labels
fig = plt.figure(figsize=(10, 10))
marker_shapes = 'o.^*' # Accomodates up to C=5
marker_colors = 'gr'

# Get sample class counts
sample_class_counts = np.array([sum(labels == j) for j in L])

# Confusion matrix compute from scratch
conf_mat = np.zeros((C, C))
for i in L: # Each decision option
    for j in L: # Each class label
        ind_ij = np.argwhere((decisions_new==i) & (labels==j))
        conf_mat[i, j] = len(ind_ij) / sample_class_counts[j] # Average over class sample count

        # True label = Marker shape; Decision = Marker Color
        marker = marker_shapes[j]
        
        

        if i != j:
            plt.plot(X[ind_ij, 0], X[ind_ij, 1], marker+'r')
        else:
            plt.plot(X[ind_ij, 0], X[ind_ij, 1], marker+'g')
            
print("Confusion matrix:")
print(conf_mat)

# # Can also compute the normalized confusion matrix from scikit-learn's method
# from sklearn.metrics import confusion_matrix
# print("Normalized Sklearn Confusion Matrix (column basis):")
# conf_mat = confusion_matrix(decisions, labels, normalize='pred')
# print(conf_mat)

prob_error = 1 - np.diag(conf_mat).dot(sample_class_counts) / N
print("Minimum Probability of Error: ", prob_error)

plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$")
plt.title("Minimum Probability of Error:  {:.3f}".format(prob_error))
plt.show()

Findings:

1. It can be seen that certain statistics and charts, such as the confusion matrix and probability error score, are not altering considerably after analysis. It might have occurred because the loss matrices for parts a and b were symmetric.

2. When compared to class 1 and 2, points with labels 3 and 4 have more wrongly anticipated points after plot observation.

3. Minimum probablity error for Part-A is 0.15259999999999996 and Part-B is 0.15300000000000002 which is very less.

4. Even in the confusion matrix, the values for the two portions are nearly identical with only a slight variation.

**Question-3**

In [ ]:
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
import math
import random
from scipy.stats import multivariate_normal
import matplotlib.patches as mpatches
from sklearn.metrics import confusion_matrix
import seaborn as sns
import string

In [ ]:
def estimate_cov_mu(data):
    '''
    Estimates the covariance matrix and mean vector for all the true class
    labels in the input data.
    Parameters
    ----------
    data: pandas.DataFrame
    Given data set
    Returns
    -------
    data_info: pandas.DataFrame
    The true labels, covariance matrics and mean vectors
    '''
    true_labels = data.index.unique().tolist()
    data_info   = pd.DataFrame(columns=['True Class Label', 'Covariance Matrix', 'Mean Vector', 'Number of Samples', 'Class Prior'])
    total_samples = 0
    for true_label in true_labels:
        temp = data.loc[true_label, :]
        #cov = np.cov(temp, bias=True)
        cov = temp.cov().to_numpy()
        mean = temp.mean(axis=0).tolist()
        n = temp.shape[0]
        total_samples = total_samples + n
        d = {'True Class Label': true_label, 'Covariance Matrix': cov, 'Mean Vector': mean, 'Number of Samples': n}
        data_info = data_info.append(d, ignore_index=True)
    data_info['Class Prior'] = data_info['Number of Samples'] / total_samples
    return data_info

In [ ]:
def plot_subset(data, subset=['x','y','z']):
    '''
    Plots the four-dimensions of the samples taken from the distribution.
    Parameters
    ----------
    data: pandas.DataFrame
    Contains the sample data
    subset: array, optional
    Plots these three values.
    Returns
    -------
    None
    '''
    markers = ['v', '^', '<', '>', '8', 's', 'p', '*', 'h', '+', 'x', 'D']
    fig = plt.figure(figsize = (5.5,5))
    fig.subplots_adjust(left=0.01, right=0.96, top=0.99, bottom=0.01, wspace=0)
    ax = plt.axes(projection ="3d")
    true_labels = data.sort_index().index.unique().tolist()
    for i, true_label in enumerate(true_labels):
        temp = data.loc[true_label, :]
        xs = temp[subset[0]].tolist()
        ys = temp[subset[1]].tolist()
        zs = temp[subset[2]].tolist()
        ax.scatter3D(xs, ys, zs, label=true_label, marker=markers[i], alpha=0.3)
    ax.set_xlabel('%s'%subset[0])
    ax.set_ylabel('%s'%subset[1])
    ax.set_zlabel('%s'%subset[2])
    ax.legend(loc='upper left', title='Class Label')
    #plt.tight_layout()
    plt.savefig('./%s_%s_%s_true_classes.pdf'%(subset[0], subset[1], subset[2]))
    plt.clf()
    return None

In [ ]:
def make_decisions(data, data_info, loss_matrix=None):
    '''
    Implement classifier and check if correct given the true
    data dsitribution knowledge. Chooses minimum risk.
    Parameters
    ----------
    data: pandas.DataFrame
    Contains the sample data
    data_info: pandas.DataFrame
    loss_matrix: array
    2d array = lambda
    Returns
    -------
    data: pandas.DataFrame
    modified data with classification and accuracy info.
    '''
    choices = []
    correct = []
    dimension_labels = data.columns.tolist()
    class_labels = data.sort_index().index.unique().tolist()
    # Create 0-1 loss matrix if none is given
    if(loss_matrix==None):
        d = max(class_labels)
        loss_matrix = np.zeros((d,d))
        for i in range(0,d):
            for j in range(0,d):
                if(i==j):
                    loss_matrix[i][j] = 0
                else:
                    loss_matrix[i][j] = 1
    print(loss_matrix)
    labels_reference = {i:class_labels[i] for i in range(0,len(class_labels))}
    for idx, row in data.iterrows():
        # Modify class label for computation
        distribution = int(row.name)
        rows = [row[dimension_label] for dimension_label in dimension_labels]
        #print(rows)
        #print(class_labels)
        args = [risk(class_label-1, rows, loss_matrix, data_info) for class_label in class_labels]
        choice = labels_reference[np.argmin(args)]
        choices.append(choice)
        #print('Choice: %d'%choice)
        #print('Correct: %d'%distribution)
        # Check if classification was correct or not
        if(choice==distribution):
            correct.append(True)
        #print('Correct!: %d'%len(correct))
        else:
            correct.append(False)
    data['ERM Classification'] = choices
    data['Correct'] = correct
    return data

In [ ]:
def risk(i , x , loss_matrix, data_info):
    '''
    Parameters
    ----------
    data_info: pandas.DataFrame
        Info on true classes in distributions.
    i: int
        The true class assigned to i
    x: 
    p: float32
        The class prior
    loss_matrix: array, optional
    '''
    risk = 0
    for j, row in data_info.iterrows():
        #  Probability, mu, sigma^2
        try:
            #print(x)
            risk = risk + loss_matrix[i][int(row['True Class Label'])-1]*row['Class Prior']*multivariate_normal.pdf(x,row['Mean Vector'],row['Covariance Matrix'])
            #print(risk)
        except np.linalg.LinAlgError:
            continue
    return risk

In [ ]:
def plot_decision_matrix(data, save_path='./decision_matrix.pdf'):
    '''
    Plots a heatmap of the decision matrix along with the values.
    Parameters
    ----------
    data: pandas.DataFrame
        Contains the sample data
    Returns
    -------
    None
    '''
    pred = data['ERM Classification'].tolist()
    act  = data.index.tolist()
    class_labels = data.sort_index().index.unique().tolist()
    confusion = confusion_matrix(act, pred, normalize='true')
    sns.heatmap(data=confusion,cmap="YlOrRd",annot=True, xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Decision')
    plt.ylabel('True Class Label')
    positions = range(0,len(class_labels))
    #plt.xticks(positions, class_labels)
    #plt.yticks(positions, class_labels)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.clf()
    plt.close()


In [ ]:
def plot_correct_classified(data, subset=['x','y','z']):
    '''
    Plots the four-dimensions of the samples taken from the distribution and if the classification was correct.
    Parameters
    ----------
    samples_path: string
        File containing the sample data
    Returns
    -------
    None
    '''
    markers = ['v', '^', '<', '>', '8', 's', 'p', '*', 'h', '+', 'x', 'D']
    fig = plt.figure(figsize = (5.5,5))
    fig.subplots_adjust(left=0.01, right=0.96, top=0.99, bottom=0.01, wspace=0)
    ax = plt.axes(projection ="3d")
    # Plot correct
    correct = data[data['Correct']==True]
    true_labels = correct.sort_index().index.unique().tolist()
    print('Number of correct classified points: %d'%correct.shape[0])
    for i, true_label in enumerate(true_labels):
        temp = correct.loc[true_label, :]
        xs = temp[subset[0]].tolist()
        ys = temp[subset[1]].tolist()
        zs = temp[subset[2]].tolist()
        ax.scatter3D(xs, ys, zs, label=true_label, marker=markers[i], alpha=0.3, color='green')
    # Plot incorrect
    correct = data[data['Correct']==False]
    true_labels = correct.sort_index().index.unique().tolist()
    print('Number of incorrect classified points: %d'%correct.shape[0])
    for i, true_label in enumerate(true_labels):
        temp = correct.loc[true_label, :]
        xs = temp[subset[0]].tolist()
        ys = temp[subset[1]].tolist()
        zs = temp[subset[2]].tolist()
        ax.scatter3D(xs, ys, zs, label=true_label, marker=markers[i], alpha=0.3, color='red')
    ax.set_xlabel('%s'%subset[0])
    ax.set_ylabel('%s'%subset[1])
    ax.set_zlabel('%s'%subset[2])
    #ax.get_legend().remove()
    green_patch = mpatches.Patch(color='green', label='Correct')
    red_patch = mpatches.Patch(color='red', label='Incorrect')
    ax.legend(handles=[green_patch, red_patch], loc='upper left', title='Classification')
    plt.savefig('./%s_%s_%s_true_class_classified_loss2.pdf'%(subset[0], subset[1], subset[2]))
    plt.clf()
    return None

In [ ]:
def write_sample_data(samples, save_path):
    '''
    Saves the sample data.
    Parameters
    ----------
    samples: numpy.array
        The generated sample data
    save_path: string
        File name and path to save the sample data
    Returns
    -------
    None
    '''
    samples.to_csv(save_path)
    
def read_sample_data(save_path):
    '''
    Read the sample data. Helper function to read data in other functions.
    Parameters
    ----------
    save_path: string
        File containing the sample data
    Returns
    -------
    samples: pandas.DataFrame
        The generated sample data
    '''
    samples = pd.read_csv(save_path, index_col=0)
    return samples

In [ ]:
if __name__=='__main__':
    # Dimensions: fixed acidity, volatile acidity, citric acid, residual sugar, chlorides, free sulfur dioxide
    wine_path = 'winequality-white.csv'
    wine_df = pd.read_csv(wine_path, delimiter=';', index_col='quality')
    print(wine_df)
    data_info = estimate_cov_mu(data=wine_df)
    print(data_info)
    wine_loss_matrix = [[0, 15, 20, 25, 30, 35, 40, 45, 50],
    [15, 0, 10, 15, 20, 25, 30, 35, 40],
    [20, 10, 0, 5, 10, 15, 20, 25, 30],
    [25, 15, 5, 0, 1, 5, 10, 15, 20],
    [30, 20, 10, 1, 0, 1, 1, 5, 10],
    [35, 25, 15, 5, 1, 0, 10, 15, 20],
    [40, 30, 20, 10, 1, 10, 0, 25, 30],
    [45, 35, 25, 15, 5, 15, 25, 0, 40],
    [50, 40, 30, 20, 10, 20, 30, 40, 0 ]]
    # alcohol, pH, residual sugar
    plot_subset(data=wine_df, subset=['alcohol', 'pH', 'residual sugar'])
    # citric acid, total sulfer dioxide, density
    plot_subset(data=wine_df, subset=['citric acid', 'total sulfur dioxide', 'density'])
    wine_df = make_decisions(data=wine_df, data_info=data_info, loss_matrix=wine_loss_matrix)
    #print(wine_df)
    plot_correct_classified(data=wine_df, subset=['alcohol', 'pH', 'residual sugar'])
    plot_correct_classified(data=wine_df, subset=['citric acid', 'total sulfur dioxide', 'density'])
    plot_decision_matrix(data=wine_df, save_path='./wine_decision_matrix_loss2.pdf')
    x_test = 'X_test.txt'
    y_test = 'y_test.txt'
    x_train = 'X_train.txt'
    y_train = 'y_train.txt'
    letters = []
    for letter_a in string.ascii_letters:
        for letter_b in string.ascii_letters[:11]:
            letters.append(letter_a+letter_b)
    letters = letters[:561]
    test_id = pd.read_csv(y_test, names=['Index'])
    test_df = pd.read_csv(x_test, delim_whitespace=True, names=letters)
    test_df = test_df.set_index(keys=test_id['Index'], drop=True)
    train_id = pd.read_csv(y_train, names=['Index'])
    train_df = pd.read_csv(x_train, delim_whitespace=True, names=letters)
    train_df = train_df.set_index(keys=train_id['Index'], drop=True)
    activity_df = test_df.append(train_df)
    activity_df = activity_df.loc[:, ['aa','ab','ac','Yh', 'Yi', 'Yj']]
    print(activity_df)
    data_info = estimate_cov_mu(data=activity_df)
    print(data_info)
    plot_subset(data=activity_df, subset=['aa', 'ab', 'ac'])
    plot_subset(data=activity_df, subset=['Yh', 'Yi', 'Yj'])
    activity_df = make_decisions(data=activity_df, data_info=data_info)
    print(activity_df)
    write_sample_data(activity_df, './activity_data.csv')
    plot_correct_classified(data=activity_df, subset=['aa', 'ab', 'ac'])
    plot_correct_classified(data=activity_df, subset=['Yh', 'Yi', 'Yj'])
    plot_decision_matrix(data=activity_df, save_path='./activity_decision_matrix.pdf')

In [ ]:
x_har_test = pd.read_fwf(x_test,delimiter=' ')
x_har_train = pd.read_fwf(x_train,delimiter=' ')

In [ ]:
x_har_test

In [ ]:
x_har_train

In [ ]:
wine_df.columns

In [ ]:
wine_exprmnt = 'winequality-white.csv'
wine_df_exprmnt = pd.read_csv(wine_exprmnt, delimiter=';')

In [ ]:
wine_df_exprmnt_y = wine_df_exprmnt['quality']
wine_df_exprmnt = wine_df_exprmnt.drop('quality',axis = 1)
wine_df_exprmnt.columns

In [ ]:
X = wine_df_exprmnt
Y = wine_df_exprmnt_y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

explained_variance = pca.explained_variance_ratio_

In [ ]:
X_train

**PCA-Wine Quality for Train Dataset**

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(X_train[:, 0], X_train[:, 1])
plt.xlabel("z1")
plt.ylabel("z2")
plt.title("PCA projections to 2D space")
plt.show()

**PCA-Wine Quality for Test Dataset**

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(X_test[:, 0], X_test[:, 1])
plt.xlabel("z1")
plt.ylabel("z2")
plt.title("PCA projections to 2D space")
plt.show()

In [ ]:
# First derive sample-based estimates of mean vector and covariance matrix:
mu_hat = np.mean(X, axis=0)
Sigma_hat = np.cov(X.T)

# Mean-subtraction is a necessary assumption for PCA, so perform this to obtain zero-mean sample set
C = X - mu_hat

# Get the eigenvectors (in U) and eigenvalues (in D) of the estimated covariance matrix
lambdas, U = np.linalg.eig(Sigma_hat)

# Get the indices from sorting lambdas in order of increasing value, with ::-1 slicing to then reverse order
idx = lambdas.argsort()[::-1]

# Extract corresponding sorted eigenvectors and eigenvalues
U = U[:, idx]
D = np.diag(lambdas[idx])

# Calculate the PC projections of zero-mean samples (in z)
Z = C.dot(U)

In [ ]:
(Z[0])

In [ ]:
# Let's see what it looks like only along the first two PCs
fig = plt.figure(figsize=(10, 10))
plt.scatter(Z[0], Z[1])
plt.xlabel("z1")
plt.ylabel("z2")
plt.title("PCA projections to 2D space")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_har_test = sc.fit_transform(x_har_test)
x_har_train = sc.transform(x_har_train)

from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
x_har_test = pca.fit_transform(x_har_test)
x_har_train = pca.transform(x_har_train)

explained_variance = pca.explained_variance_ratio_

**PCA- HAR Dataset Test Data**

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(x_har_test[:, 0], x_har_test[:, 1])
plt.xlabel("z1")
plt.ylabel("z2")
plt.title("PCA projections to 2D space")
plt.show()

**PCA- HAR Dataset Train Data**

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(x_har_train[:, 0], x_har_train[:, 1])
plt.xlabel("z1")
plt.ylabel("z2")
plt.title("PCA projections to 2D space")
plt.show()